## Main

In [ ]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from geopy.location import Location
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [ ]:
geolocator = Nominatim(user_agent="Urban-Topology-Analysis-Service1")

In [ ]:
residents = pd.read_csv(
    "../../data/Кол_жильцов_подомовой_список.csv"
    )

In [ ]:
residents

In [ ]:
new_data = pd.DataFrame({"Адрес": [], "Геопозиция": [], "Кол-во жильцов": []})

for i in range(residents.shape[0]):
    street = residents.iloc[i, 1]
    city = ", Санкт-Петербург"
    if "КИНГИСЕППСКАЯ" in street or "РЫБАЦКАЯ УЛ., ЛОМОНОСОВСКИЙ РАЙОН" in street or "МАРЬИНО, ЛОМОНОСОВСКИЙ РАЙОН" in street:
        city = ""
    row = [
            str(residents.iloc[i, 2]) + ", " + residents.iloc[i, 1] + city,
            None,
            residents.iloc[i, 3:].sum()
          ]
    new_data = pd.concat([pd.DataFrame([row], columns=new_data.columns), new_data], ignore_index=True)

print(new_data)

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
new_data["Геопозиция"] = new_data["Адрес"].progress_apply(geocode)
new_data["Геопозиция"] = new_data["Геопозиция"].apply(lambda loc: (loc.latitude, loc.longitude) if loc else None)

In [ ]:
points = pd.read_csv("../data/result/Residents.csv")

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def apply_geocode(street):
    location = geocode(street)
    return (location.latitude, location.longitude) if location else None

points.loc[~points["Геопозиция"].str.startswith("("), "Геопозиция"] = points.loc[~points["Геопозиция"].str.startswith("("), "Геопозиция"].progress_apply(apply_geocode)

In [ ]:
points.loc[points["Геопозиция"].isnull(), "Геопозиция"] = points.loc[points["Геопозиция"].isnull(), "Адрес"].progress_apply(apply_geocode)

In [ ]:
points.to_csv("../../data/result/Residents.csv")

In [ ]:
null_in_points = points.loc[points["Геопозиция"].isnull()]

In [ ]:
null_in_points

In [ ]:

new_data.to_csv("Residents.csv")

## Refactor null data

In [ ]:
null_in_new_data = new_data.loc[new_data["Геопозиция"].isnull()]

In [ ]:
null_in_new_data.to_csv("./null_data.csv")

In [ ]:
for i in range(27417, 27456):
    # parts = new_data.iloc[i, 0].split(",")
    # new_data.iloc[i, 0] = ",".join(parts[:-1])
    new_data.iloc[i, 0] = new_data.iloc[i, 0].replace("ПЕРВОГО", "1")
    print(new_data.iloc[i, 0])

In [ ]:
def apply_geocode(street):
    location = geocode(street)
    return (location.latitude, location.longitude) if location else None
new_data.loc[new_data["Геопозиция"].isnull(), "Геопозиция"] = new_data.loc[new_data["Геопозиция"].isnull(), "Адрес"].progress_apply(apply_geocode)

## Data analysis

In [ ]:
streets = []
area = []

for i in range(residents.shape[0]):
    area.append(residents.iloc[i, 0])
    streets.append(residents.iloc[i, 1])

area = list(set(area))
set_streets = list(set(streets))

In [ ]:
len(set_streets)

### Parse all streets with digits

In [ ]:
import re

In [ ]:
streets_with_digits = []
for street in set_streets:
    match = re.fullmatch(".*\d+.*", street)
    if match:
        streets_with_digits.append(street)

len(streets_with_digits)

In [ ]:
streets_with_digits

### Sort streets by length

In [ ]:
big_streets = []
for street in set_streets:
    if (len(street.split(" ")) > 2):
        big_streets.append(street)

len(big_streets)

In [ ]:
big_streets

In [ ]:
standart_streets = []
for street in set_streets:
    if (len(street.split(" ")) == 2):
        standart_streets.append(street)

len(standart_streets)

In [ ]:
standart_streets

### Take the last part of streets

In [ ]:
last_part = []
for street in set_streets:
    part = street.split(" ")[-1]
    if part.startswith("УЛ."):
        last_part.append(part[3:])
    elif part.startswith("ПЕР."):
        last_part.append(part[4:])
    else:
        last_part.append(part)

last_part = list(set(last_part))
len(last_part)

In [ ]:
sorted(last_part)

## Take data for one area

In [ ]:
for a in area:
    print(residents.loc[residents["Район"] == a].shape)


In [ ]:
residents["Район"].nunique()

In [ ]:
data_for_pushkin = pd.DataFrame({"Адрес": [], "Геопозиция": [], "Кол-во жильцов": []})
pushkin_residents = residents.loc[residents["Район"] == "ПУШКИНСКИЙ"]

for i in range(pushkin_residents.shape[0]):
    row = [
            pushkin_residents.iloc[i, 2] + ", " + pushkin_residents.iloc[i, 1] + ", Санкт-Петербург",
            None,
            pushkin_residents.iloc[i, 3:].sum()
          ]
    data_for_pushkin = pd.concat([pd.DataFrame([row], columns=data_for_pushkin.columns), data_for_pushkin], ignore_index=True)

In [ ]:
data_for_pushkin

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
data_for_pushkin["Геопозиция"] = data_for_pushkin["Адрес"].progress_apply(geocode)
data_for_pushkin["Геопозиция"] = data_for_pushkin["Геопозиция"].apply(lambda loc: (loc.latitude, loc.longitude) if loc else None)

In [ ]:
data_for_pushkin.to_csv("Pushkin.csv", sep=";")

In [ ]:
data_for_admiralt = pd.DataFrame({"Адрес": [], "Геопозиция": [], "Кол-во жильцов": []})
admiralt_residents = residents.loc[residents["Район"] == "АДМИРАЛТЕЙСКИЙ"]

for i in range(admiralt_residents.shape[0]):
    row = [
            admiralt_residents.iloc[i, 2] + ", " + admiralt_residents.iloc[i, 1] + ", Санкт-Петербург",
            None,
            admiralt_residents.iloc[i, 3:].sum()
          ]
    data_for_admiralt = pd.concat([pd.DataFrame([row], columns=data_for_admiralt.columns), data_for_admiralt], ignore_index=True)

In [ ]:
data_for_admiralt

In [ ]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
data_for_admiralt["Геопозиция"] = data_for_admiralt["Адрес"].progress_apply(geocode)
data_for_admiralt["Геопозиция"] = data_for_admiralt["Геопозиция"].apply(lambda loc: (loc.latitude, loc.longitude) if loc else None)

In [ ]:
vc_for_admiralt = data_for_admiralt["Геопозиция"].value_counts()

In [ ]:
vc_for_admiralt[vc_for_admiralt > 1]